In [59]:
import pandas as pd

data_dir = "./Other_Datasets/ANERcorp/"
name = "dev"

bad_words = {'ARTICLE', 'REVISIT', 'ARTICLE', 'article', 'REVIIST', 'ARTICE', 'ARTICLE', 'FIX', 'ID', ' ', 'REVISSIT', 'REVISIIT'}

sentences = []
sentences_labels = []

with open(data_dir + f"{name}.txt", 'r') as file:
    sentence = []
    each_word_label = []
    for i, line in enumerate(file):
        # Check if the line is not empty
        if line.strip():
            parts = line.split(' ')
            # Check if parts has enough elements
            if len(parts) > 1:
                word, label = parts[0], parts[1].strip()
                # Check if the word is not in the set of bad words
                if label not in bad_words:
                    sentence.append(word)
                    each_word_label.append(label)

                # If a the word is found a period, it indicates the end of a sentence
                if '.' == word:
                    if sentence:
                        sentences.append(' '.join(sentence))
                        sentences_labels.append(each_word_label)
                        sentence = []
                        each_word_label = []

# Now you have sentences and their corresponding labels
df = pd.DataFrame({'sentence': sentences, 'labels': sentences_labels})


In [60]:
new_df = pd.DataFrame()

for i in range(len(df)):
    words = df['sentence'][i].split(' ')
    labels = df['labels'][i]
    num_words = len(words)
    
    for j in range(num_words):
        new_row = pd.DataFrame({'label': [labels[j]], 'word': [words[j]], 'sentence': df['sentence'][i]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)


In [61]:
new_df['label'].unique()

array(['O', 'B-LOC', 'I-LOC', 'B-ORG', 'B-PERS', 'I-PERS', 'B-MISC',
       'I-ORG', 'I-MISC'], dtype=object)

In [62]:
new_df.loc[new_df['label'] == '']

,label,word,sentence


In [58]:
new_df.to_csv(data_dir + f"{name}.csv", index=False, header=False)